In [1]:
import torch
from my_cryptopals_tools import hex_to_base64, xor_hex_strings, xor_byte_strings, chi_squared_score
import base64


In [3]:
hex_string = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
print(hex_to_base64(hex_string))

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


In [2]:
hex_string1 = "1c0111001f010100061a024b53535009181c"
hex_string2 = "686974207468652062756c6c277320657965"

xor_hex_strings(hex_string1, hex_string2)

'746865206b696420646f6e277420706c6179'

In [2]:
hex_string1 = "1c0111001f010100061a024b53535009181c"
hex_string2 = "686974207468652062756c6c277320657965"

xor_hex_strings(hex_string1, hex_string2)

byte_string1 = bytes.fromhex(hex_string1)
byte_string2 = bytes.fromhex(hex_string2)

print (f'String 1 in bytes is {byte_string1}')
print (f'String 2 in bytes is {byte_string2}')
print (f'Xor in bytes is {xor_byte_strings(byte_string1, byte_string2)}')
print (f'Xor solution in hex is {xor_hex_strings(hex_string1, hex_string2)}')

String 1 in bytes is b'\x1c\x01\x11\x00\x1f\x01\x01\x00\x06\x1a\x02KSSP\t\x18\x1c'
String 2 in bytes is b"hit the bull's eye"
Xor in bytes is b"the kid don't play"
Xor solution in hex is 746865206b696420646f6e277420706c6179


In [2]:
hex_string = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'


best_score = 9999999.
for i in range(88,95):
    byte_string1 = bytes.fromhex(hex_string)
    byte_string2 = i.to_bytes()

    #print (f'Xor in bytes with {i} is {xor_byte_strings(byte_string1, byte_string2)}')
    #print (f'Xor in hex is {xor_byte_strings(byte_string1, byte_string2).decode('utf-8')}')
    raw_string = xor_byte_strings(byte_string1, byte_string2).decode('latin-1')
    print (raw_string)
    score = chi_squared_score(raw_string)
    if score < best_score:
        best_score = score
        best_char = i
        best_string = raw_string
    print (f'score {i} is {score}')

print (f'Best string was: "{best_string}"')
print ('Best char was: ', best_char)
print ('Best score was: ', best_score)


Cooking MC's like a pound of bacon
score 88 is 61.98055038379752
Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno
score 89 is 201.14471125862227
Ammikle"OA%q"nkig"c"rmwlf"md"`caml
score 90 is 144.06710815803274
@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm
score 91 is 175.76283535141496
Gkkomjc$IG#w$hmoa$e$tkqj`$kb$fegkj
score 92 is 206.2190473203588
Fjjnlkb%HF"v%iln`%d%ujpka%jc%gdfjk
score 93 is 212.51754224123897
Eiimoha&KE!u&jomc&g&vishb&i`&dgeih
score 94 is 128.3412608482029
Best string was: "Cooking MC's like a pound of bacon"
Best char was:  88
Best score was:  61.98055038379752


In [12]:
from collections import Counter
import math

def score_string_by_letter_frequency_improved(string: str, letter_freqs: dict) -> float:
    """
    Scores a string based on letter frequency, normalizes by length, and penalizes repetition.

    :param string: The string to score.
    :param letter_freqs: A dictionary of letter frequencies.
    :return: A likelihood score for the string.
    """
    string = string.lower()  # Normalize to lowercase
    total_score = 0.0
    char_count = Counter(string)  # Count occurrences of each character
    length = len(string)
    
    if length == 0:
        return 0  # Avoid division by zero

    # Add letter frequency scores for valid characters
    for char, count in char_count.items():
        if char in letter_freqs:
            total_score += letter_freqs[char] * count

    # Normalize the score by length
    normalized_score = total_score / length

    # Penalize low diversity using Shannon entropy
    probabilities = [count / length for count in char_count.values()]
    entropy = -sum(p * math.log2(p) for p in probabilities if p > 0)

    # Combine normalized score with entropy (weighted for balance)
    final_score = normalized_score * 0.7 + entropy * 0.3

    return final_score

def compare_strings_by_likelihood_improved(string1: str, string2: str, letter_freqs: dict) -> str:
    """
    Compares two strings to determine which is more likely an English sentence.

    :param string1: The first string.
    :param string2: The second string.
    :param letter_freqs: A dictionary of letter frequencies.
    :return: The string more likely to be English text.
    """
    score1 = score_string_by_letter_frequency_improved(string1, letter_freqs)
    score2 = score_string_by_letter_frequency_improved(string2, letter_freqs)

    return string1 if score1 > score2 else string2

# Example usage
letter_freqs = {'e': 0.112, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
                't': 0.0665, 's': 0.0573, 'l': 0.0548}

string1 = "Cooking MC's like a pound of bacon"
string2 = "@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm"

more_likely = compare_strings_by_likelihood_improved(string1, string2, letter_freqs)
print(f"The more likely English sentence is: '{more_likely}'")


The more likely English sentence is: '@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm'


In [14]:
from collections import Counter

def chi_squared_score(s, letter_freqs):
    # Normalize the string: lowercase and keep only alphabetic characters
    s = s.lower()
    s = ''.join(filter(str.isalpha, s))
    total_chars = len(s)
    if total_chars == 0:
        return float('inf')  # Empty strings are penalized

    # Count observed frequencies of each letter in the string
    observed_counts = Counter(s)

    # Calculate chi-squared statistic
    chi_squared = 0
    for letter, expected_freq in letter_freqs.items():
        observed = observed_counts.get(letter, 0)
        expected = expected_freq * total_chars
        chi_squared += (observed - expected) ** 2 / expected if expected > 0 else 0

    return chi_squared

def find_most_english_like(strings, letter_freqs):
    # Score each string using chi-squared statistic (lower is better)
    scores = {s: chi_squared_score(s, letter_freqs) for s in strings}
    return min(scores, key=scores.get), scores

# Test inputs
strings = [
    "Cooking MC's like a pound of bacon",
    "@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm",
    "eeeeeeeeeeeeeeeeeeee",
    "kjhifykboipjepbudoipo"
]

letter_freqs = {
    'e': 0.1116, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
    't': 0.0665, 's': 0.0573, 'l': 0.0548, 'c': 0.0453, 'u': 0.0363,
    'd': 0.0384, 'p': 0.0316, 'm': 0.0301, 'h': 0.0300, 'g': 0.0247
}

most_english_like, all_scores = find_most_english_like(strings, letter_freqs)
print("Most English-like string:", most_english_like)
print("Scores:", all_scores)

Most English-like string: Cooking MC's like a pound of bacon
Scores: {"Cooking MC's like a pound of bacon": 14.067738958676314, '@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm': 29.311344507822206, 'eeeeeeeeeeeeeeeeeeee': 155.89946953405018, 'kjhifykboipjepbudoipo': 21.320287079675367}


In [16]:
from collections import Counter
import math

def kl_divergence_score(s, letter_freqs):
    # Normalize the string: lowercase and keep only alphabetic characters
    s = s.lower()
    s = ''.join(filter(str.isalpha, s))
    total_chars = len(s)
    if total_chars == 0:
        return float('inf')  # Empty strings are penalized

    # Count occurrences of each letter in the string
    observed_counts = Counter(s)
    
    # Calculate observed probabilities
    observed_probs = {char: count / total_chars for char, count in observed_counts.items()}
    
    # Calculate KL divergence
    kl_divergence = 0
    for char, observed_prob in observed_probs.items():
        expected_prob = letter_freqs.get(char, 1e-10)  # Use a small value to avoid log(0)
        kl_divergence += observed_prob * math.log2(observed_prob / expected_prob)
    
    return kl_divergence

def find_most_english_like_kl(strings, letter_freqs):
    # Score each string using KL divergence (lower is better)
    scores = {s: kl_divergence_score(s, letter_freqs) for s in strings}
    return min(scores, key=scores.get), scores

# Test inputs
strings = [
    "Cooking MC's like a pound of bacon",
    "@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm",
    "eeeeeeeeeeeeeeeeeeee",
    "kjhifykboipjepbudoipo"
]

letter_freqs = {
    'e': 0.1116, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
    't': 0.0665, 's': 0.0573, 'l': 0.0548, 'c': 0.0453, 'u': 0.0363,
    'd': 0.0384, 'p': 0.0316, 'm': 0.0301, 'h': 0.0300, 'g': 0.0247
}

most_english_like, all_scores = find_most_english_like_kl(strings, letter_freqs)
print("Most English-like string by KL divergence:", most_english_like)
print("KL divergence scores:", all_scores)
  

Most English-like string by KL divergence: eeeeeeeeeeeeeeeeeeee
KL divergence scores: {"Cooking MC's like a pound of bacon": 7.948107645911927, '@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm': 9.19850376242986, 'eeeeeeeeeeeeeeeeeeee': 3.1635910677202617, 'kjhifykboipjepbudoipo': 11.859082152195056}


In [18]:
from collections import Counter
import string

def chi_squared_score(s, letter_freqs):
    """
    Computes the chi-squared score for how closely the letter distribution of a string matches expected frequencies.
    """
    # Normalize the input string and count letters
    s = s.lower()
    counts = Counter(char for char in s if char in letter_freqs)
    total_letters = sum(counts.values())
    
    if total_letters == 0:
        return float('inf')  # No valid letters, penalize heavily
    
    chi_squared = 0
    for letter, expected_freq in letter_freqs.items():
        observed = counts.get(letter, 0)
        expected = total_letters * expected_freq
        chi_squared += (observed - expected) ** 2 / expected if expected > 0 else 0

    return chi_squared

def find_most_english_string(strings, letter_freqs):
    """
    Finds the string that is most likely an English sentence based on chi-squared scoring.
    Lower scores indicate a closer match to English letter frequencies.
    """
    scores = {s: chi_squared_score(s, letter_freqs) for s in strings}
    return min(scores, key=scores.get), scores

# Provided strings and letter frequencies
strings = [
    "Cooking MC's like a pound of bacon",
    "@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm",
    "eeeeeeeeeeeeeeeeeeee",
    "kjhifykboipjepbudoipo",
    "Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno",
    "--)+,%b e1b.+)'b#b2-7,&b-$b #!-,"
]

letter_freqs = {'e': 0.1116, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
                't': 0.0665, 's': 0.0573, 'l': 0.0548, 'c': 0.0453, 'u': 0.0363,
                'd': 0.0384, 'p': 0.0316, 'm': 0.0301, 'h': 0.0300, 'g': 0.0247, ' ': 0.2}

# Find the most likely English sentence
most_english, scores = find_most_english_string(strings, letter_freqs)

print(f"The most likely English sentence is: '{most_english}'")
print("Chi-squared scores for each string:")
for s, score in scores.items():
    print(f"'{s}': {score}")


The most likely English sentence is: '--)+,%b e1b.+)'b#b2-7,&b-$b #!-,'
Chi-squared scores for each string:
'Cooking MC's like a pound of bacon': 14.569451995548476
'@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm': 44.6940863639843
'eeeeeeeeeeeeeeeeeeee': 159.89946953405018
'kjhifykboipjepbudoipo': 35.5822483594756
'Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno': 21.00841194715189
'--)+,%b e1b.+)'b#b2-7,&b-$b #!-,': 6.756724492234169


In [ ]:
Let's pretend we have five strings:

string1 = "Cooking MC's like a pound of bacon"
string2 = "@llhjmd#N@$p#ojhf#b#slvmg#le#ablm"
string3 = "eeeeeeeeeeeeeeeeeeeeeee"
string4 = "kjhifykboipjepbudoipo"
string5 = "Bnnjhof!LB&r!mhjd!!qntoe!ng!c`bno"
string6 = "--)+,%b e1b.+)'b#b2-7,&b-$b #!-,"

How do we write a python function that determines which of these strings is most likely to be a real english sentence using the letter frequencies to score them:

letter_freqs = {'e': 0.1116, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
                    't': 0.0665, 's': 0.0573, 'l': 0.0548, 'c': 0.0453, 'u': 0.0363,
                    'd': 0.0384, 'p': 0.0316, 'm': 0.0301, 'h': 0.0300, 'g': 0.0247, ' ': 0.2}

In [25]:
from collections import Counter

def chi_squared_score(string, letter_freqs):
    # Normalize: Convert to lowercase and count all characters
    normalized = ''.join(c.lower() if c.lower() in letter_freqs else '#' for c in string)
    
    # Count occurrences of each character
    total_chars = len(normalized)
    if total_chars == 0:  # Avoid division by zero
        return float('inf')
    
    observed_counts = Counter(normalized)
    
    # Calculate chi-squared score
    chi_squared = 0
    for char, expected_freq in letter_freqs.items():
        observed = observed_counts.get(char, 0)
        expected = total_chars * expected_freq
        chi_squared += ((observed - expected) ** 2) / expected if expected > 0 else 0

    # Add penalty for uncommon characters
    uncommon_count = observed_counts.get('#', 0)
    chi_squared += uncommon_count * 10  # Large penalty for each uncommon character
    
    return chi_squared

def most_likely_sentence(strings, letter_freqs):
    # Score each string using the chi-squared method
    scores = {string: chi_squared_score(string, letter_freqs) for string in strings}
    best_string = min(scores, key=scores.get)  # Lower chi-squared is better
    return best_string, scores

# Letter frequencies
letter_freqs = {
    'e': 0.1116, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
    't': 0.0665, 's': 0.0573, 'l': 0.0548, 'c': 0.0453, 'u': 0.0363,
    'd': 0.0384, 'p': 0.0316, 'm': 0.0301, 'h': 0.0300, 'g': 0.0247, 
    ' ': 0.2  # Spaces are very common in English sentences
}

# Strings to analyze
strings = [
    "Cooking MC's like a pound of bacon",
    "@llhjmd#N@$p#ojhf#b#slvmg#le#ablm",
    "eeeeeeeeeeeeeeeeeeeeeee",
    "kjhifykboipjepbudoipo",
    "Bnnjhof!LB&r!mhjd!!qntoe!ng!c`bno",
    "--)+,%b e1b.+)'b#b2-7,&b-$b #!-,"
]

# Get the most likely sentence
best_string, scores = most_likely_sentence(strings, letter_freqs)

print(f"Most likely English sentence: {best_string}")
print(f"Chi-squared scores: {scores}")






Most likely English sentence: Cooking MC's like a pound of bacon
Chi-squared scores: {"Cooking MC's like a pound of bacon": 93.50934564365471, '@llhjmd#N@$p#ojhf#b#slvmg#le#ablm': 191.61574146023435, 'eeeeeeeeeeeeeeeeeeeeeee': 183.8843899641577, 'kjhifykboipjepbudoipo': 105.52028707967537, 'Bnnjhof!LB&r!mhjd!!qntoe!ng!c`bno': 221.35628349433256, "--)+,%b e1b.+)'b#b2-7,&b-$b #!-,": 318.00581792114696}


In [9]:
from math import log2
from collections import Counter

def entropy_score(s, letter_freqs):
    # Normalize string: convert to lowercase and filter relevant characters
    #normalized = ''.join(c.lower() for c in string if c.lower() in letter_freqs)
    normalized = ''.join(c.lower() if c.lower() in letter_freqs else '#' for c in s)
    
    # Count occurrences of each letter
    total_chars = len(normalized)
    if total_chars == 0:  # Avoid division by zero
        return float('inf')
    
    observed_counts = Counter(normalized)
    
    # Calculate observed frequencies
    observed_freqs = {char: count / total_chars for char, count in observed_counts.items()}
    
    # Calculate entropy deviation
    entropy = 0
    for char, expected_freq in letter_freqs.items():
        observed_freq = observed_freqs.get(char, 0)
        if observed_freq > 0:
            entropy += -observed_freq * log2(expected_freq)
    
    # Penalize strings with missing or uncommon characters
    uncommon_count = total_chars - sum(observed_counts[char] for char in letter_freqs if char in observed_counts)
    entropy += uncommon_count * 10  # Large penalty for uncommon characters
    
    return entropy

def most_likely_sentence(strings, letter_freqs):
    # Score each string using the entropy method
    scores = {string: entropy_score(string, letter_freqs) for string in strings}
    best_string = min(scores, key=scores.get)  # Lower entropy is better
    return best_string, scores

# Letter frequencies
letter_freqs = {
    'e': 0.1116, 'a': 0.0850, 'r': 0.0758, 'i': 0.0754, 'o': 0.0716, 
    't': 0.0665, 's': 0.0573, 'l': 0.0548, 'c': 0.0453, 'u': 0.0363,
    'd': 0.0384, 'p': 0.0316, 'm': 0.0301, 'h': 0.0300, 'g': 0.0247, 
    ' ': 0.2, '#': 0.001  # Spaces are very common in English sentences
}

# Strings to analyze
strings = [
    "Cooking MC's like a pound of bacon",
    "@llhjmd#N@$p#ojhf#b#slvmg#le#ablm",
    "eeeeeeeeeeeeeeeeeeeeeee",
    "kjhifykboipjepbudoipo",
    "Bnnjhof!LB&r!mhjd!!qntoe!ng!c`bno",
    "--)+,%b e1b.+)'b#b2-7,&b-$b #!-,"
]

# Get the most likely sentence
best_string, scores = most_likely_sentence(strings, letter_freqs)

print(f"Most likely English sentence: {best_string}")
print(f"Entropy scores: {scores}")

Most likely English sentence: eeeeeeeeeeeeeeeeeeeeeee
Entropy scores: {"Cooking MC's like a pound of bacon": 5.205421462464042, '@llhjmd#N@$p#ojhf#b#slvmg#le#ablm': 7.1320074015606725, 'eeeeeeeeeeeeeeeeeeeeeee': 3.1635910677202617, 'kjhifykboipjepbudoipo': 6.42794072639369, 'Bnnjhof!LB&r!mhjd!!qntoe!ng!c`bno': 7.739125069761785, "--)+,%b e1b.+)'b#b2-7,&b-$b #!-,": 9.275474734771734}


In [8]:
log2(0.01)

-6.643856189774724